In [406]:
from sage.all import *
from sage.rings.number_field.number_field import CyclotomicField
from sage.rings.integer_ring import ZZ
from sage.modules.free_module_element import vector
from sage.numerical.mip import MixedIntegerLinearProgram as MILP

In [423]:
def K(p):
    assert p.is_prime()
    KK.<z> = CyclotomicField(p)
    return KK, z

def theta(z, j, p):
    return z^j + z^(p-j)

def K_plus(KK, z, p):
    assert p.is_prime()
    G = KK.galois_group()
    (s,) = G.gens()
    sigma_neg_one = s ^ (p // 2)
    H = G.subgroup([sigma_neg_one])
    Kp, f = H.fixed_field()
    (tau,) = Kp.galois_group().gens()
    t = theta(z, 1, p)
#     print(f(tau(t)), tau(t), ",", t, theta(z, 10, p), t^10, sep="\n")
    for c in range(1, p):
        if f(tau(t)) == theta(z, c, p):
            break
    c = ZZ(c)
    d = ZZ(mod(c, p)^(-1))
    return Kp, tau, c, d

def eta(ell, z):
    return z^ell * (1 - z) / (1 - z ^ (2 * ell + 1))

def conj(a, i, tau):
    return (tau ^ i)(a)

def maxconj(a, tau, p):
    conjs = [conj(a, i, tau) for i in range((p - 1) // 2)]
    abs_conjs = [sgn(c) * c for c in conjs]
#     print([(a, float(real(a))) for a in abs_conjs])
    i, a = max(enumerate([real(a) for a in abs_conjs]), key=lambda x: x[1])
    return i+1, abs_conjs[i]

def Xi(z, tau, p):
    m = (p - 1) // 2
    etas = [eta(ell, z) for ell in range(1, m)]
    return (matrix([
        [log(abs(conj(e, i, tau))) for e in etas]
        for i in range(1, m)
    ]), etas)
    
def Xi_i(X, i, p):
    row_i = X[i]
    A = matrix([row_i]*X.nrows())
    A[i] = 0
    return X - A

def eps_i(M, X, etas, i, p):
    m = (p-1) // 2
    
    X_i = Xi_i(X, i, p)
    # X_i_inv = X_i.inverse()
    
    L_i = vector([-log(m-2)]*(m-1), RR)
    L_i[i] = log(M/2)
    
    ilp = MILP()
    k_var = ilp.new_variable(integer=True, nonnegative=False)
    ks = [k_var[f"k{j}"] for j in range(1,m)]
    objective = sum(k * log(abs(e)) for (k,e) in zip(ks, etas))
    # print("minimize:", objective)
    ilp.set_objective(-objective)
    for x, l in zip(X_i, L_i):
        c = sum(a * k for (a,k) in zip(x, ks)) <= l
        # print(c)
        ilp.add_constraint(c)
    # print()
    ilp.solve()
    
    ks_solved = [ilp.get_values(k, convert=ZZ, tolerance=0.0001) for k in ks]
    epsilon = prod(e ^ k for (k,e) in zip(ks_solved, etas))
    
    return epsilon

def E_prime(M, X, etas, p):
    m = (p-1) // 2
    return [eps_i(M, X, etas, i, p) for i in range(m-1)]

def w(i, d, p):
    return ZZ((d^i * (p-1)//2) % p)

def E(Ep, tau, d, p):
    m = (p - 1) // 2
    ts = [theta(z, w(k, d, p), p) for k in range(1,m)]
    eps = [sgn(conj(e, k, tau)) * e for k, e in enumerate(Ep, start=1)]
    return eps + [t * e for (t,e) in zip(ts, eps)]

In [438]:
p = 11
KK, z = K(p)
G = KK.galois_group()
M = 16
print("zeta:", complex(z), end="\n\n")
Kp, tau, c, d = K_plus(KK, z, p)
print("K_p^plus:", Kp, end="\n\n")
print("tau:", tau, end="\n\n")
print("c:", c, end="\n\n")
print("d:", d, end="\n\n")
X, etas = Xi(z, tau, p)
print("Xi:", X, (X.ncols(), X.nrows()), sep="\n", end="\n\n")
print("etas:", [float(real(e)) for e in etas], end="\n\n")
Ep = E_prime(M, X, etas, p)
E_ = E(Ep, tau, d, p)
print("E_M:")
for e in E_:
    print(float(real(e)), "=", e)
print()
m = (p-1) // 2
print("Checks (1):")
for j in range(1,m):
    j_ = j - 1
    print(f"{real(maxconj(E_[j_], tau, p)[1])} == {real(conj(E_[j_], j, tau))}")
    print(f"{real(maxconj(E_[j_+m-1], tau, p)[1])} == {real(-conj(E_[j_+m-1], j, tau))}")
    assert maxconj(E_[j_], tau, p)[1] == conj(E_[j_], j, tau), f"{maxconj(E_[j_], tau, p)}, {float(real(maxconj(E_[j_], tau, p)[1]))} != {conj(E_[j_], j, tau)}, {float(real(conj(E_[j_], j, tau)))}, j={j}"
    assert maxconj(E_[j_+m-1], tau, p)[1] == -conj(E_[j_+m-1], j, tau), f"{maxconj(E_[j_+m-1], tau, p)}, {float(real(maxconj(E_[j_+m-1], tau, p)[1]))} != {-conj(E_[j_+m-1], j, tau)}, {float(real(-conj(E_[j_+m-1], j, tau)))}, j={j}"
print("Checks (2):")
for e in E_:
    print(f"{real(sum(abs(conj(e, i, tau)) for i in range(m)))} <= {real(2 * maxconj(e, tau, p)[1])}")
    assert real(sum(abs(conj(e, i, tau)) for i in range(m))) <= real(2 * maxconj(e, tau, p)[1]), f"{real(sum(conj(e, i, tau) for i in range(m)))} <= {real(2 * maxconj(e, tau, p)[1])}"
print("Checks (3):")
print(f"{real(max(maxconj(e, tau, p)[1] for e in E_))} <= {M}")
assert real(max(maxconj(e, tau, p)[1] for e in E_)) <= M
print("All Checks Passed")

zeta: (0.8412535328311812+0.5406408174555976j)

K_p^plus: Number Field in z0 with defining polynomial x^5 - x^4 - 4*x^3 + 3*x^2 + 3*x - 1 with z0 = -1.682507065662363?

tau: (1,3,4,2,5)

c: 2

d: 6

Xi:
[ -0.604769430145586   0.651796894033141  -0.334954935470212  -0.520284982030078]
[   1.17208187606322   0.520284982030075 -0.0844844481155112   0.185330046559863]
[  0.334954935470213  -0.185330046559862   0.986751829503353  -0.269814494675378]
[ 0.0844844481155153   0.269814494675375   0.604769430145589    1.25656632417873]
(4, 4)

etas: [0.37278559777179265, 0.2846296765465707, 0.3097214678905702, 0.5211085581132027]

E_M:
0.2173207689761658 = -2*z^9 - 2*z^8 - z^7 - z^4 - 2*z^3 - 2*z^2
0.10610584411500468 = z^8 - z^7 + z^6 + z^5 - z^4 + z^3 + 1
-0.052395572669121204 = -2*z^9 + z^7 - z^6 - z^5 + z^4 - 2*z^2 + 1
0.12323116899741104 = 2*z^9 + 2*z^7 + z^6 + z^5 + 2*z^4 + 2*z^2 + 3
-0.061855940180539415 = 3*z^9 + 3*z^8 + 2*z^7 - z^6 - z^5 + 2*z^4 + 3*z^3 + 3*z^2 - 1
-0.13896910190607148 =

In [439]:
def approx_alg(M, p):
    KK, z = K(p)
    G = KK.galois_group()
    Kp, tau, c, d = K_plus(KK, z, p)
    X, etas = Xi(z, tau, p)
    Eps = [E_prime(2^k, X, etas, p) for k in range(int(log(M, 2))+1)]
    Es = [E(Ep, tau, d, p) for (k, Ep) in zip(range(int(log(M, 2))+1), Eps)]
    m = (p-1) // 2
    for (k, E_) in zip(range(int(log(M, 2))+1), Es):
        print(k,":",max([round(float(real(e)), 3) for e in E_]), [round(float(real(e)), 3) for e in E_])
    
    def alg(omega):
        nonlocal Es
        a = 0
#         Es = Es[-2:]
        for k, E in zip(range(int(log(M, 2))+1), Es):
            print("\nE:", k)
            while real(a - omega) * (-1)^k < 0:
                ell, _ = maxconj(a, tau, p)
                al = conj(a, ell, tau)
                ell_ = ell - 1
                e = sgn(al)
                print("sgn", e)
                if e == -1:
                    print("-", ell_, ell)
                    ep = E[ell_]
                else:
                    print("+", ell_ + m - 2, ell)
                    ep = E[ell_+m-1]
                a += (-1)^k * ep
                print(float(real(a)), "=", a)
        return a
    
    def alt(omega):
        nonlocal Es
        a = 0
        A = 0
        for k, E in zip(range(int(log(M, 2))+1), Es):
            print("\nE:", k)
            while real(a - omega) < 0:
                ell, _ = maxconj(a, tau, p)
                al = conj(a, ell, tau)
                ell_ = ell - 1
                e = sgn(al)
                print("sgn", e)
                if e == -1:
                    print("-", ell_, ell)
                    ep = E[ell_]
                else:
                    print("+", ell_ + m - 2, ell)
                    ep = E[ell_+m-1]
                a += ep
                print(float(real(a)), "=", a)
            A += (-1)^k * a
            omega = a - omega
            a = 0
        return A
    
    return alg, alt

In [440]:
alg, alt = approx_alg(32, 11)

0 : 22082.049 [11429.268, -11300.592, -5580.286, 13124.491, -3253.109, 14800.628, -4636.269, 22082.049]
1 : 1788.956 [345.174, 544.132, -491.935, 1063.268, -98.247, -712.662, -408.715, 1788.956]
2 : 77.877 [-30.429, -47.969, 23.687, 46.286, 8.661, 62.825, 19.68, 77.877]
3 : 1.576 [-5.148, -1.204, -3.823, -4.911, 1.465, 1.576, -3.176, -8.263]
4 : 0.217 [0.217, 0.106, -0.052, 0.123, -0.062, -0.139, -0.044, 0.207]
5 : 0.007 [-0.026, 0.003, 0.001, -0.013, 0.007, -0.003, 0.001, -0.022]


In [ ]:
omega = 0.3
a = alt(omega)
print("="*10)
print(float(real(a)), "=", a)
print(float(real(omega - a)))


E: 0
sgn 0
+ 3 1
-3253.108838079187 = 252*z^9 + 582*z^8 + 885*z^7 + 1065*z^6 + 1065*z^5 + 885*z^4 + 582*z^3 + 252*z^2 - 94
sgn -1
- 0 1
8176.1591059713455 = -633*z^9 - 1462*z^8 - 2224*z^7 - 2677*z^6 - 2677*z^5 - 2224*z^4 - 1462*z^3 - 633*z^2 + 236

E: 1
sgn 0
+ 3 1
-98.24672391418132 = 8*z^9 + 18*z^8 + 27*z^7 + 32*z^6 + 32*z^5 + 27*z^4 + 18*z^3 + 8*z^2 - 3
sgn -1
- 0 1
246.9271353481942 = -19*z^9 - 44*z^8 - 67*z^7 - 81*z^6 - 81*z^5 - 67*z^4 - 44*z^3 - 19*z^2 + 7
sgn -1
- 0 1
592.1009946105697 = -46*z^9 - 106*z^8 - 161*z^7 - 194*z^6 - 194*z^5 - 161*z^4 - 106*z^3 - 46*z^2 + 17
sgn 1
+ 3 1
493.8542706963884 = -38*z^9 - 88*z^8 - 134*z^7 - 162*z^6 - 162*z^5 - 134*z^4 - 88*z^3 - 38*z^2 + 14
sgn -1
- 0 1
839.0281299587639 = -65*z^9 - 150*z^8 - 228*z^7 - 275*z^6 - 275*z^5 - 228*z^4 - 150*z^3 - 65*z^2 + 24
sgn -1
- 0 1
1184.2019892211395 = -92*z^9 - 212*z^8 - 322*z^7 - 388*z^6 - 388*z^5 - 322*z^4 - 212*z^3 - 92*z^2 + 34
sgn 1
+ 3 1
1085.9552653069582 = -84*z^9 - 194*z^8 - 295*z^7 - 356*z^6 - 3

sgn -1
- 0 1
-469.7748777443844 = 36*z^9 + 81*z^8 + 126*z^7 + 153*z^6 + 153*z^5 + 126*z^4 + 81*z^3 + 36*z^2 - 18
sgn 1
+ 3 1
-461.1138472903698 = 36*z^9 + 80*z^8 + 124*z^7 + 150*z^6 + 150*z^5 + 124*z^4 + 80*z^3 + 36*z^2 - 18
sgn -1
- 0 1
-491.54296683650955 = 38*z^9 + 85*z^8 + 132*z^7 + 160*z^6 + 160*z^5 + 132*z^4 + 85*z^3 + 38*z^2 - 19
sgn -1
- 0 1
-521.9720863826494 = 40*z^9 + 90*z^8 + 140*z^7 + 170*z^6 + 170*z^5 + 140*z^4 + 90*z^3 + 40*z^2 - 20
sgn 1
+ 3 1
-513.3110559286347 = 40*z^9 + 89*z^8 + 138*z^7 + 167*z^6 + 167*z^5 + 138*z^4 + 89*z^3 + 40*z^2 - 20
sgn -1
- 0 1
-543.7401754747746 = 42*z^9 + 94*z^8 + 146*z^7 + 177*z^6 + 177*z^5 + 146*z^4 + 94*z^3 + 42*z^2 - 21
sgn -1
- 0 1
-574.1692950209143 = 44*z^9 + 99*z^8 + 154*z^7 + 187*z^6 + 187*z^5 + 154*z^4 + 99*z^3 + 44*z^2 - 22
sgn 1
+ 3 1
-565.5082645668997 = 44*z^9 + 98*z^8 + 152*z^7 + 184*z^6 + 184*z^5 + 152*z^4 + 98*z^3 + 44*z^2 - 22
sgn -1
- 0 1
-595.9373841130395 = 46*z^9 + 103*z^8 + 160*z^7 + 194*z^6 + 194*z^5 + 160*z^4 + 103*z

sgn -1
- 0 1
-1683.4177350625885 = 130*z^9 + 291*z^8 + 452*z^7 + 548*z^6 + 548*z^5 + 452*z^4 + 291*z^3 + 130*z^2 - 65
sgn -1
- 0 1
-1713.846854608728 = 132*z^9 + 296*z^8 + 460*z^7 + 558*z^6 + 558*z^5 + 460*z^4 + 296*z^3 + 132*z^2 - 66
sgn 1
+ 3 1
-1705.1858241547136 = 132*z^9 + 295*z^8 + 458*z^7 + 555*z^6 + 555*z^5 + 458*z^4 + 295*z^3 + 132*z^2 - 66
sgn -1
- 0 1
-1735.6149437008535 = 134*z^9 + 300*z^8 + 466*z^7 + 565*z^6 + 565*z^5 + 466*z^4 + 300*z^3 + 134*z^2 - 67
sgn -1
- 0 1
-1766.0440632469931 = 136*z^9 + 305*z^8 + 474*z^7 + 575*z^6 + 575*z^5 + 474*z^4 + 305*z^3 + 136*z^2 - 68
sgn 1
+ 3 1
-1757.3830327929786 = 136*z^9 + 304*z^8 + 472*z^7 + 572*z^6 + 572*z^5 + 472*z^4 + 304*z^3 + 136*z^2 - 68
sgn -1
- 0 1
-1787.8121523391183 = 138*z^9 + 309*z^8 + 480*z^7 + 582*z^6 + 582*z^5 + 480*z^4 + 309*z^3 + 138*z^2 - 69
sgn -1
- 0 1
-1818.2412718852581 = 140*z^9 + 314*z^8 + 488*z^7 + 592*z^6 + 592*z^5 + 488*z^4 + 314*z^3 + 140*z^2 - 70
sgn 1
+ 3 1
-1809.5802414312434 = 140*z^9 + 313*z^8 + 486*z

sgn -1
- 0 1
-2875.2925032886674 = 222*z^9 + 497*z^8 + 772*z^7 + 936*z^6 + 936*z^5 + 772*z^4 + 497*z^3 + 222*z^2 - 111
sgn -1
- 0 1
-2905.721622834807 = 224*z^9 + 502*z^8 + 780*z^7 + 946*z^6 + 946*z^5 + 780*z^4 + 502*z^3 + 224*z^2 - 112
sgn 1
+ 3 1
-2897.0605923807925 = 224*z^9 + 501*z^8 + 778*z^7 + 943*z^6 + 943*z^5 + 778*z^4 + 501*z^3 + 224*z^2 - 112
sgn -1
- 0 1
-2927.4897119269326 = 226*z^9 + 506*z^8 + 786*z^7 + 953*z^6 + 953*z^5 + 786*z^4 + 506*z^3 + 226*z^2 - 113
sgn -1
- 0 1
-2957.9188314730723 = 228*z^9 + 511*z^8 + 794*z^7 + 963*z^6 + 963*z^5 + 794*z^4 + 511*z^3 + 228*z^2 - 114
sgn 1
+ 3 1
-2949.2578010190573 = 228*z^9 + 510*z^8 + 792*z^7 + 960*z^6 + 960*z^5 + 792*z^4 + 510*z^3 + 228*z^2 - 114
sgn -1
- 0 1
-2979.686920565197 = 230*z^9 + 515*z^8 + 800*z^7 + 970*z^6 + 970*z^5 + 800*z^4 + 515*z^3 + 230*z^2 - 115
sgn -1
- 0 1
-3010.116040111337 = 232*z^9 + 520*z^8 + 808*z^7 + 980*z^6 + 980*z^5 + 808*z^4 + 520*z^3 + 232*z^2 - 116
sgn 1
+ 3 1
-3001.455009657322 = 232*z^9 + 519*z^8 + 

sgn 1
+ 3 1
-4036.7381519686064 = 312*z^9 + 698*z^8 + 1084*z^7 + 1314*z^6 + 1314*z^5 + 1084*z^4 + 698*z^3 + 312*z^2 - 156
sgn -1
- 0 1
-4067.167271514746 = 314*z^9 + 703*z^8 + 1092*z^7 + 1324*z^6 + 1324*z^5 + 1092*z^4 + 703*z^3 + 314*z^2 - 157
sgn -1
- 0 1
-4097.596391060886 = 316*z^9 + 708*z^8 + 1100*z^7 + 1334*z^6 + 1334*z^5 + 1100*z^4 + 708*z^3 + 316*z^2 - 158
sgn 1
+ 3 1
-4088.9353606068717 = 316*z^9 + 707*z^8 + 1098*z^7 + 1331*z^6 + 1331*z^5 + 1098*z^4 + 707*z^3 + 316*z^2 - 158
sgn -1
- 0 1
-4119.364480153012 = 318*z^9 + 712*z^8 + 1106*z^7 + 1341*z^6 + 1341*z^5 + 1106*z^4 + 712*z^3 + 318*z^2 - 159
sgn -1
- 0 1
-4149.793599699151 = 320*z^9 + 717*z^8 + 1114*z^7 + 1351*z^6 + 1351*z^5 + 1114*z^4 + 717*z^3 + 320*z^2 - 160
sgn 1
+ 3 1
-4141.132569245136 = 320*z^9 + 716*z^8 + 1112*z^7 + 1348*z^6 + 1348*z^5 + 1112*z^4 + 716*z^3 + 320*z^2 - 160
sgn -1
- 0 1
-4171.5616887912765 = 322*z^9 + 721*z^8 + 1120*z^7 + 1358*z^6 + 1358*z^5 + 1120*z^4 + 721*z^3 + 322*z^2 - 161
sgn -1
- 0 1
-4201.99080

...

sgn 1
+ 3 1
-219370.03059496073 = 16930*z^9 + 37913*z^8 + 58896*z^7 + 71414*z^6 + 71414*z^5 + 58896*z^4 + 37913*z^3 + 16930*z^2 - 8465
sgn -1
- 0 1
-219400.45971450684 = 16932*z^9 + 37918*z^8 + 58904*z^7 + 71424*z^6 + 71424*z^5 + 58904*z^4 + 37918*z^3 + 16932*z^2 - 8466
sgn -1
- 0 1
-219430.88883405298 = 16934*z^9 + 37923*z^8 + 58912*z^7 + 71434*z^6 + 71434*z^5 + 58912*z^4 + 37923*z^3 + 16934*z^2 - 8467
sgn 1
+ 3 1
-219422.22780359897 = 16934*z^9 + 37922*z^8 + 58910*z^7 + 71431*z^6 + 71431*z^5 + 58910*z^4 + 37922*z^3 + 16934*z^2 - 8467
sgn -1
- 0 1
-219452.6569231451 = 16936*z^9 + 37927*z^8 + 58918*z^7 + 71441*z^6 + 71441*z^5 + 58918*z^4 + 37927*z^3 + 16936*z^2 - 8468
sgn -1
- 0 1
-219483.08604269125 = 16938*z^9 + 37932*z^8 + 58926*z^7 + 71451*z^6 + 71451*z^5 + 58926*z^4 + 37932*z^3 + 16938*z^2 - 8469
sgn 1
+ 3 1
-219474.42501223725 = 16938*z^9 + 37931*z^8 + 58924*z^7 + 71448*z^6 + 71448*z^5 + 58924*z^4 + 37931*z^3 + 16938*z^2 - 8469
sgn -1
- 0 1
-219504.8541317834 = 16940*z^9 + 37936*

sgn -1
- 0 1
-220435.74285681813 = 17012*z^9 + 38097*z^8 + 59182*z^7 + 71761*z^6 + 71761*z^5 + 59182*z^4 + 38097*z^3 + 17012*z^2 - 8506
sgn -1
- 0 1
-220466.17197636428 = 17014*z^9 + 38102*z^8 + 59190*z^7 + 71771*z^6 + 71771*z^5 + 59190*z^4 + 38102*z^3 + 17014*z^2 - 8507
sgn 1
+ 3 1
-220457.51094591024 = 17014*z^9 + 38101*z^8 + 59188*z^7 + 71768*z^6 + 71768*z^5 + 59188*z^4 + 38101*z^3 + 17014*z^2 - 8507
sgn -1
- 0 1
-220487.9400654564 = 17016*z^9 + 38106*z^8 + 59196*z^7 + 71778*z^6 + 71778*z^5 + 59196*z^4 + 38106*z^3 + 17016*z^2 - 8508
sgn -1
- 0 1
-220518.36918500255 = 17018*z^9 + 38111*z^8 + 59204*z^7 + 71788*z^6 + 71788*z^5 + 59204*z^4 + 38111*z^3 + 17018*z^2 - 8509
sgn 1
+ 3 1
-220509.70815454854 = 17018*z^9 + 38110*z^8 + 59202*z^7 + 71785*z^6 + 71785*z^5 + 59202*z^4 + 38110*z^3 + 17018*z^2 - 8509
sgn -1
- 0 1
-220540.13727409465 = 17020*z^9 + 38115*z^8 + 59210*z^7 + 71795*z^6 + 71795*z^5 + 59210*z^4 + 38115*z^3 + 17020*z^2 - 8510
sgn -1
- 0 1
-220570.56639364082 = 17022*z^9 + 3812

sgn 1
+ 3 1
-221471.0259991294 = 17092*z^9 + 38276*z^8 + 59460*z^7 + 72098*z^6 + 72098*z^5 + 59460*z^4 + 38276*z^3 + 17092*z^2 - 8546
sgn -1
- 0 1
-221501.45511867557 = 17094*z^9 + 38281*z^8 + 59468*z^7 + 72108*z^6 + 72108*z^5 + 59468*z^4 + 38281*z^3 + 17094*z^2 - 8547
sgn -1
- 0 1
-221531.88423822168 = 17096*z^9 + 38286*z^8 + 59476*z^7 + 72118*z^6 + 72118*z^5 + 59476*z^4 + 38286*z^3 + 17096*z^2 - 8548
sgn 1
+ 3 1
-221523.22320776767 = 17096*z^9 + 38285*z^8 + 59474*z^7 + 72115*z^6 + 72115*z^5 + 59474*z^4 + 38285*z^3 + 17096*z^2 - 8548
sgn -1
- 0 1
-221553.65232731382 = 17098*z^9 + 38290*z^8 + 59482*z^7 + 72125*z^6 + 72125*z^5 + 59482*z^4 + 38290*z^3 + 17098*z^2 - 8549
sgn -1
- 0 1
-221584.08144685993 = 17100*z^9 + 38295*z^8 + 59490*z^7 + 72135*z^6 + 72135*z^5 + 59490*z^4 + 38295*z^3 + 17100*z^2 - 8550
sgn 1
+ 3 1
-221575.42041640598 = 17100*z^9 + 38294*z^8 + 59488*z^7 + 72132*z^6 + 72132*z^5 + 59488*z^4 + 38294*z^3 + 17100*z^2 - 8550
sgn -1
- 0 1
-221605.8495359521 = 17102*z^9 + 38299*

-222506.30914144072 = 17172*z^9 + 38455*z^8 + 59738*z^7 + 72435*z^6 + 72435*z^5 + 59738*z^4 + 38455*z^3 + 17172*z^2 - 8586
sgn -1
- 0 1
-222536.73826098684 = 17174*z^9 + 38460*z^8 + 59746*z^7 + 72445*z^6 + 72445*z^5 + 59746*z^4 + 38460*z^3 + 17174*z^2 - 8587
sgn -1
- 0 1
-222567.167380533 = 17176*z^9 + 38465*z^8 + 59754*z^7 + 72455*z^6 + 72455*z^5 + 59754*z^4 + 38465*z^3 + 17176*z^2 - 8588
sgn 1
+ 3 1
-222558.50635007897 = 17176*z^9 + 38464*z^8 + 59752*z^7 + 72452*z^6 + 72452*z^5 + 59752*z^4 + 38464*z^3 + 17176*z^2 - 8588
sgn -1
- 0 1
-222588.9354696251 = 17178*z^9 + 38469*z^8 + 59760*z^7 + 72462*z^6 + 72462*z^5 + 59760*z^4 + 38469*z^3 + 17178*z^2 - 8589
sgn -1
- 0 1
-222619.36458917125 = 17180*z^9 + 38474*z^8 + 59768*z^7 + 72472*z^6 + 72472*z^5 + 59768*z^4 + 38474*z^3 + 17180*z^2 - 8590
sgn 1
+ 3 1
-222610.70355871724 = 17180*z^9 + 38473*z^8 + 59766*z^7 + 72469*z^6 + 72469*z^5 + 59766*z^4 + 38473*z^3 + 17180*z^2 - 8590
sgn -1
- 0 1
-222641.13267826336 = 17182*z^9 + 38478*z^8 + 59774*z

sgn -1
- 0 1
-223572.02140329813 = 17254*z^9 + 38639*z^8 + 60024*z^7 + 72782*z^6 + 72782*z^5 + 60024*z^4 + 38639*z^3 + 17254*z^2 - 8627
sgn -1
- 0 1
-223602.45052284427 = 17256*z^9 + 38644*z^8 + 60032*z^7 + 72792*z^6 + 72792*z^5 + 60032*z^4 + 38644*z^3 + 17256*z^2 - 8628
sgn 1
+ 3 1
-223593.78949239023 = 17256*z^9 + 38643*z^8 + 60030*z^7 + 72789*z^6 + 72789*z^5 + 60030*z^4 + 38643*z^3 + 17256*z^2 - 8628
sgn -1
- 0 1
-223624.2186119364 = 17258*z^9 + 38648*z^8 + 60038*z^7 + 72799*z^6 + 72799*z^5 + 60038*z^4 + 38648*z^3 + 17258*z^2 - 8629
sgn -1
- 0 1
-223654.64773148252 = 17260*z^9 + 38653*z^8 + 60046*z^7 + 72809*z^6 + 72809*z^5 + 60046*z^4 + 38653*z^3 + 17260*z^2 - 8630
sgn 1
+ 3 1
-223645.9867010285 = 17260*z^9 + 38652*z^8 + 60044*z^7 + 72806*z^6 + 72806*z^5 + 60044*z^4 + 38652*z^3 + 17260*z^2 - 8630
sgn -1
- 0 1
-223676.41582057468 = 17262*z^9 + 38657*z^8 + 60052*z^7 + 72816*z^6 + 72816*z^5 + 60052*z^4 + 38657*z^3 + 17262*z^2 - 8631
sgn -1
- 0 1
-223706.84494012076 = 17264*z^9 + 38662